In [1]:
import cv2
import os
import numpy as np
from random import shuffle
from tqdm import tqdm
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

In [2]:
# TRAIN_DIR = 'J:/SONGS/LAP DOWNLOAD/GEETA/Compressed/Data of saudia/Data of saudia'
# TRAIN_DIR = 'C:/Users/DELL/SkyDrive/Documents/data2/train_data'
TRAIN_DIR = 'E:/full stack python/Traine_DATA/Traine_DATA'
# TEST_DIR = 'J:/SONGS/LAP DOWNLOAD/GEETA/Compressed/Data of saudia/tes'
# TEST_DIR = 'C:/Users/DELL/SkyDrive/Documents/data2/test_data'
TEST_DIR = 'E:/full stack python/Test_DATA/Test_DATA'
IMG_SIZE = 50
LR = 1e-3
 
 
'''Setting up the model which will help with tensorflow models'''
MODEL_NAME = 'Autistic-{}-{}.model'.format(LR, '6conv-basic')

In [4]:
def label_img(img):
    word_label = img.split('.')#[-3]
    s=word_label[0]
 # DIY One hot encoder
#     if 'happy' in s : 
#         return [1, 0, 0, 0]
#     elif 'sad' in  s: 
#         return [0, 1, 0, 0]
#     elif 'angry' in  s: 
#         return [0, 0, 1, 0] 
#     elif 'normal' in  s: 
#         return [0, 0, 0, 1] 
    if 'appy' in s : 
        return [1, 0, 0, 0]
    elif 'sad' in  s: 
        return [0, 1, 0, 0]
    elif 'ngry' in  s: 
        return [0, 0, 1, 0] 
    elif (('ormal' in  s ) or ('eutral' in  s)): 
        return [0, 0, 0, 1]

In [5]:
def create_train_data():
    # Creating an empty list where we should the store the training data
    # after a little preprocessing of the data
    training_data = []
 
    # tqdm is only used for interactive loading
    # loading the training data
    for img in tqdm(os.listdir(TRAIN_DIR)):
 
        # labeling the images
        label = label_img(img)
 
        path = os.path.join(TRAIN_DIR, img)
 
        # loading the image from the path and then converting them into
        # greyscale for easier covnet prob
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
 
        # resizing the image for processing them in the covnet
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
 
        # final step-forming the training data list with numpy array of the images
        training_data.append([np.array(img), np.array(label)])
 
    # shuffling of the training data to preserve the random state of our data
    shuffle(training_data)
 
    # saving our trained data for further uses if required
    np.save('train_data.npy', training_data)
    return training_data
 

In [6]:
def process_test_data():
    testing_data = []
    for img in tqdm(os.listdir(TEST_DIR)):
        path = os.path.join(TEST_DIR, img)
        img_num = img.split('.')[0]
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        testing_data.append([np.array(img), img_num])
         
    shuffle(testing_data)
    np.save('test_data.npy', testing_data)
    return testing_data
 

In [7]:
'''Running the training and the testing in the dataset for our model'''
train_data = create_train_data()
test_data = process_test_data()
 

100%|████████████████████████████████████████| 299/299 [00:28<00:00, 10.44it/s]


In [210]:
# test_data.shape

In [8]:
train_data[0][0]
type(train_data)
len(train_data)

1228

In [9]:
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
 
import tensorflow as tf
tf.reset_default_graph()
convnet = input_data(shape =[None, IMG_SIZE, IMG_SIZE, 1], name ='input')
 
convnet = conv_2d(convnet, 32, 5, activation ='relu')
convnet = max_pool_2d(convnet, 5)
 
convnet = conv_2d(convnet, 64, 5, activation ='relu')
convnet = max_pool_2d(convnet, 5)
 
convnet = conv_2d(convnet, 128, 5, activation ='relu')
convnet = max_pool_2d(convnet, 5)
 
convnet = conv_2d(convnet, 64, 5, activation ='relu')
convnet = max_pool_2d(convnet, 5)
 
convnet = conv_2d(convnet, 32, 5, activation ='relu')
convnet = max_pool_2d(convnet, 5)
 
convnet = fully_connected(convnet, 1024, activation ='relu')
convnet = dropout(convnet, 0.3)
 
convnet = fully_connected(convnet, 4, activation ='softmax')
convnet = regression(convnet, optimizer ='adam', learning_rate = LR,
      loss ='categorical_crossentropy', name ='targets')
 
model = tflearn.DNN(convnet, tensorboard_dir ='log')

C:\Users\DELL\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


curses is not supported on this machine (please install/reinstall curses for an optimal experience)
Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.


In [10]:
# random classifier model implementation

from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
# clf = RandomForestClassifier(max_depth=2, random_state=0)
clf = RandomForestClassifier(n_estimators=100)
RandomX = np.array([i[0] for i in train_data]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
RandomY = [i[1] for i in train_data]
# RandomX=RandomX.reshape(27,2500)
RandomX=RandomX.reshape(len(train_data),IMG_SIZE*IMG_SIZE)
print("RandomX shape",RandomX.shape)

#  svm y editing
svm_y = []
for val in RandomY:
    if np.argmax(val) == 0:
#         str_label ='Happy'
        svm_y.append(0)
    elif np.argmax(val) == 1:
#         str_label ='sad'
        svm_y.append(1)
    elif np.argmax(val) == 2:
#         str_label ='Angry'
        svm_y.append(2)
    elif np.argmax(val) == 3:
#         str_label ='normal'
        svm_y.append(3)


RandomY_numpy_array = np.array(RandomY)
# type(RandomX)
# print((RandomY))
# print((RandomY.shape))
print((RandomY_numpy_array.shape))
# print((RandomY.ndim))
clf.fit(RandomX,RandomY)
s_v_m = svm.SVC(gamma=0.001,C=100)
s_v_m.fit(RandomX,svm_y)

RandomX shape (1228, 2500)
(1228, 4)


SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [11]:
len(train_data)
int(len(train_data) * .3)

368

In [12]:
# Splitting the testing data and training data
split_value=int(len(train_data) * .3) # 30 /70 split 
train = train_data[:-split_value]
test = train_data[-split_value:]


In [13]:
# X-Features & Y-Labels
 
X = np.array([i[0] for i in train]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
Y = [i[1] for i in train]
test_x = np.array([i[0] for i in test]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
test_y = [i[1] for i in test]
# print("test_y",len(test_y))

In [14]:
model_details = model.fit({'input': X}, {'targets': Y}, n_epoch = 10, 
    validation_set =({'input': test_x}, {'targets': test_y}), 
    snapshot_step = 2000, show_metric = True, run_id = MODEL_NAME)
model.save(MODEL_NAME)

Training Step: 139  | total loss: 0.95516 | time: 3.267s
| Adam | epoch: 010 | loss: 0.95516 - acc: 0.5590 -- iter: 832/860
Training Step: 140  | total loss: 0.98774 | time: 4.543s
| Adam | epoch: 010 | loss: 0.98774 - acc: 0.5469 | val_loss: 1.10116 - val_acc: 0.5190 -- iter: 860/860
--
INFO:tensorflow:C:\Users\DELL\Autistic-0.001-6conv-basic.model is not in all_model_checkpoint_paths. Manually adding it.


In [19]:
'''Testing the data'''
from sklearn import metrics
import matplotlib.pyplot as plt
# if you need to create the data:
# test_data = process_test_data()
# if you already have some saved:
test_data = np.load('test_data.npy')
 
# fig = plt.figure()
Label_OF_test_data = [] 
Predicted_label = []
Predicted_label_RandomForest = []
b = True


for num, data in enumerate(test_data):
    # cat: [1, 0]
    # dog: [0, 1]
     
    img_num = data[1]
    img_data = data[0]
#     print(img_num)
#     if 'happy' in img_num : 
#         Label_OF_test_data.append(0)
#     elif 'sad' in  img_num: 
#         Label_OF_test_data.append(1)
#     elif 'angry' in  img_num: 
#         Label_OF_test_data.append(2)
#     elif 'normal' in  img_num: 
#         Label_OF_test_data.append(3)
#             return [1, 0, 0, 0]
    if 'appy' in img_num : 
        Label_OF_test_data.append(0)
    elif 'sad' in  img_num: 
        Label_OF_test_data.append(1)
    elif 'ngry' in  img_num: 
        Label_OF_test_data.append(2) 
    elif (('ormal' in  img_num ) or ('eutral' in  img_num)): 
        Label_OF_test_data.append(3)
        
    
#     y = fig.add_subplot(4, 5, num + 1)
    orig = img_data
    data = img_data.reshape(IMG_SIZE, IMG_SIZE, 1)
 
    # model_out = model.predict([data])[0]
#     print(data.shape)
#     print(data.ndim)
    
    dataRC=data.reshape(1,IMG_SIZE*IMG_SIZE)
#     print(dataRC.shape)
#     print(dataRC.ndim)
#     if b:
#         break
    model_out = model.predict([data])[0]
#     print(np.argmax(model_out))
    if np.argmax(model_out) == 0:
#         str_label ='Happy'
        Predicted_label.append(0)
    elif np.argmax(model_out) == 1:
#         str_label ='sad'
        Predicted_label.append(1)
    elif np.argmax(model_out) == 2:
#         str_label ='Angry'
        Predicted_label.append(2)
    elif np.argmax(model_out) == 3:
#         str_label ='normal'
        Predicted_label.append(3)
    
        

score=metrics.accuracy_score(Label_OF_test_data, Predicted_label)
print(Label_OF_test_data)
print(Predicted_label)
print('Accuracy cnn : {}'.format(score))


[3, 2, 1, 0, 2, 3, 1, 0, 1, 3, 3, 2, 2, 0, 2, 3, 1, 1, 1, 2, 0, 1, 1, 3, 2, 1, 0, 2, 2, 0, 0, 2, 2, 2, 2, 3, 2, 3, 1, 1, 3, 3, 3, 2, 2, 3, 3, 1, 2, 2, 3, 3, 2, 1, 0, 1, 3, 2, 3, 2, 3, 2, 1, 1, 1, 0, 1, 2, 2, 0, 2, 0, 0, 2, 1, 3, 0, 3, 1, 1, 0, 2, 3, 0, 3, 3, 3, 1, 1, 1, 2, 3, 2, 0, 3, 3, 2, 0, 2, 0, 2, 0, 3, 3, 1, 1, 3, 0, 3, 0, 1, 0, 1, 1, 3, 3, 0, 1, 3, 0, 2, 3, 2, 2, 3, 3, 3, 3, 3, 2, 2, 1, 2, 1, 1, 0, 0, 2, 3, 1, 1, 0, 0, 1, 2, 0, 3, 0, 2, 1, 2, 3, 2, 0, 3, 2, 2, 1, 0, 3, 2, 1, 0, 0, 0, 1, 2, 2, 3, 0, 0, 0, 0, 1, 0, 0, 2, 1, 0, 2, 2, 1, 3, 1, 3, 2, 0, 3, 1, 0, 0, 2, 2, 0, 1, 3, 2, 1, 0, 0, 3, 3, 0, 1, 2, 0, 1, 3, 2, 1, 2, 2, 1, 3, 1, 0, 1, 2, 2, 2, 1, 3, 2, 0, 2, 3, 3, 3, 3, 3, 2, 3, 0, 2, 1, 1, 1, 0, 2, 0, 1, 3, 1, 3, 3, 0, 1, 2, 1, 1, 3, 3, 3, 3, 1, 3, 0, 0, 0, 2, 0, 0, 1, 2, 0, 0, 0, 1, 3, 0, 3, 3, 1, 0, 3, 1, 2, 3, 0, 0, 1, 0, 0, 0, 3, 2, 3, 0, 2, 0, 1, 2, 0, 2, 3, 3, 3, 2, 1]
[0, 2, 1, 3, 2, 3, 1, 2, 0, 2, 0, 0, 0, 0, 2, 3, 1, 1, 1, 3, 0, 1, 1, 2, 2, 2, 0, 2, 2, 0, 0, 2, 0, 3,

In [20]:
# RANDOM CLASSIFIER PREDICTION

test_Xrc = np.array([i[0] for i in test_data]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
print(test_Xrc.ndim)
print(test_Xrc.shape)
test_Xrc=test_Xrc.reshape(len(test_Xrc),IMG_SIZE*IMG_SIZE)
print(test_Xrc.ndim)
print(test_Xrc.shape)
# *************************IMPLEMENTING rANDOM fOREST MODEL ******************************************
pred_rc=clf.predict(test_Xrc)
# print(pred_rc)
final_pred_rc=[]
for val in pred_rc:
    if np.argmax(val) == 0:
#         str_label ='Happy'
        final_pred_rc.append(0)
    elif np.argmax(val) == 1:
#         str_label ='sad'
        final_pred_rc.append(1)
    elif np.argmax(val) == 2:
#         str_label ='Angry'
        final_pred_rc.append(2)
    elif np.argmax(val) == 3:
#         str_label ='normal'
        final_pred_rc.append(3)
# RC ACCURACY CHECK

score=metrics.accuracy_score(Label_OF_test_data, final_pred_rc)
print(Label_OF_test_data)
print(final_pred_rc)
print('Accuracy RANDOM CLASSIFIER: {}'.format(score))

4
(299, 50, 50, 1)
2
(299, 2500)
[3, 2, 1, 0, 2, 3, 1, 0, 1, 3, 3, 2, 2, 0, 2, 3, 1, 1, 1, 2, 0, 1, 1, 3, 2, 1, 0, 2, 2, 0, 0, 2, 2, 2, 2, 3, 2, 3, 1, 1, 3, 3, 3, 2, 2, 3, 3, 1, 2, 2, 3, 3, 2, 1, 0, 1, 3, 2, 3, 2, 3, 2, 1, 1, 1, 0, 1, 2, 2, 0, 2, 0, 0, 2, 1, 3, 0, 3, 1, 1, 0, 2, 3, 0, 3, 3, 3, 1, 1, 1, 2, 3, 2, 0, 3, 3, 2, 0, 2, 0, 2, 0, 3, 3, 1, 1, 3, 0, 3, 0, 1, 0, 1, 1, 3, 3, 0, 1, 3, 0, 2, 3, 2, 2, 3, 3, 3, 3, 3, 2, 2, 1, 2, 1, 1, 0, 0, 2, 3, 1, 1, 0, 0, 1, 2, 0, 3, 0, 2, 1, 2, 3, 2, 0, 3, 2, 2, 1, 0, 3, 2, 1, 0, 0, 0, 1, 2, 2, 3, 0, 0, 0, 0, 1, 0, 0, 2, 1, 0, 2, 2, 1, 3, 1, 3, 2, 0, 3, 1, 0, 0, 2, 2, 0, 1, 3, 2, 1, 0, 0, 3, 3, 0, 1, 2, 0, 1, 3, 2, 1, 2, 2, 1, 3, 1, 0, 1, 2, 2, 2, 1, 3, 2, 0, 2, 3, 3, 3, 3, 3, 2, 3, 0, 2, 1, 1, 1, 0, 2, 0, 1, 3, 1, 3, 3, 0, 1, 2, 1, 1, 3, 3, 3, 3, 1, 3, 0, 0, 0, 2, 0, 0, 1, 2, 0, 0, 0, 1, 3, 0, 3, 3, 1, 0, 3, 1, 2, 3, 0, 0, 1, 0, 0, 0, 3, 2, 3, 0, 2, 0, 1, 2, 0, 2, 3, 3, 3, 2, 1]
[0, 0, 1, 0, 2, 3, 1, 0, 0, 0, 0, 2, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1,

In [21]:
#  implemrnting SVM 
pred_svm=[]
pred_svm=s_v_m.predict(test_Xrc)
# print(pred_rc)
#  accuracy scheck svm

score=metrics.accuracy_score(Label_OF_test_data, pred_svm)
print(Label_OF_test_data)
print(pred_svm)
print('Accuracy SVM: {}'.format(score))

[3, 2, 1, 0, 2, 3, 1, 0, 1, 3, 3, 2, 2, 0, 2, 3, 1, 1, 1, 2, 0, 1, 1, 3, 2, 1, 0, 2, 2, 0, 0, 2, 2, 2, 2, 3, 2, 3, 1, 1, 3, 3, 3, 2, 2, 3, 3, 1, 2, 2, 3, 3, 2, 1, 0, 1, 3, 2, 3, 2, 3, 2, 1, 1, 1, 0, 1, 2, 2, 0, 2, 0, 0, 2, 1, 3, 0, 3, 1, 1, 0, 2, 3, 0, 3, 3, 3, 1, 1, 1, 2, 3, 2, 0, 3, 3, 2, 0, 2, 0, 2, 0, 3, 3, 1, 1, 3, 0, 3, 0, 1, 0, 1, 1, 3, 3, 0, 1, 3, 0, 2, 3, 2, 2, 3, 3, 3, 3, 3, 2, 2, 1, 2, 1, 1, 0, 0, 2, 3, 1, 1, 0, 0, 1, 2, 0, 3, 0, 2, 1, 2, 3, 2, 0, 3, 2, 2, 1, 0, 3, 2, 1, 0, 0, 0, 1, 2, 2, 3, 0, 0, 0, 0, 1, 0, 0, 2, 1, 0, 2, 2, 1, 3, 1, 3, 2, 0, 3, 1, 0, 0, 2, 2, 0, 1, 3, 2, 1, 0, 0, 3, 3, 0, 1, 2, 0, 1, 3, 2, 1, 2, 2, 1, 3, 1, 0, 1, 2, 2, 2, 1, 3, 2, 0, 2, 3, 3, 3, 3, 3, 2, 3, 0, 2, 1, 1, 1, 0, 2, 0, 1, 3, 1, 3, 3, 0, 1, 2, 1, 1, 3, 3, 3, 3, 1, 3, 0, 0, 0, 2, 0, 0, 1, 2, 0, 0, 0, 1, 3, 0, 3, 3, 1, 0, 3, 1, 2, 3, 0, 0, 1, 0, 0, 0, 3, 2, 3, 0, 2, 0, 1, 2, 0, 2, 3, 3, 3, 2, 1]
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 

In [22]:
# implementing Logistic regression
from sklearn.linear_model import LogisticRegression

# all parameters not specified are set to their defaults# all pa 
# default solver is incredibly slow thats why we change it
# solver = 'lbfgs'
logisticRegr = LogisticRegression(solver = 'lbfgs')
logisticRegr.fit(RandomX, svm_y)
pred_logReg=logisticRegr.predict(test_Xrc)
score=metrics.accuracy_score(Label_OF_test_data, pred_logReg)
print(Label_OF_test_data)
print(pred_logReg)
print('Accuracy LogisticRegression : {}'.format(score))

[3, 2, 1, 0, 2, 3, 1, 0, 1, 3, 3, 2, 2, 0, 2, 3, 1, 1, 1, 2, 0, 1, 1, 3, 2, 1, 0, 2, 2, 0, 0, 2, 2, 2, 2, 3, 2, 3, 1, 1, 3, 3, 3, 2, 2, 3, 3, 1, 2, 2, 3, 3, 2, 1, 0, 1, 3, 2, 3, 2, 3, 2, 1, 1, 1, 0, 1, 2, 2, 0, 2, 0, 0, 2, 1, 3, 0, 3, 1, 1, 0, 2, 3, 0, 3, 3, 3, 1, 1, 1, 2, 3, 2, 0, 3, 3, 2, 0, 2, 0, 2, 0, 3, 3, 1, 1, 3, 0, 3, 0, 1, 0, 1, 1, 3, 3, 0, 1, 3, 0, 2, 3, 2, 2, 3, 3, 3, 3, 3, 2, 2, 1, 2, 1, 1, 0, 0, 2, 3, 1, 1, 0, 0, 1, 2, 0, 3, 0, 2, 1, 2, 3, 2, 0, 3, 2, 2, 1, 0, 3, 2, 1, 0, 0, 0, 1, 2, 2, 3, 0, 0, 0, 0, 1, 0, 0, 2, 1, 0, 2, 2, 1, 3, 1, 3, 2, 0, 3, 1, 0, 0, 2, 2, 0, 1, 3, 2, 1, 0, 0, 3, 3, 0, 1, 2, 0, 1, 3, 2, 1, 2, 2, 1, 3, 1, 0, 1, 2, 2, 2, 1, 3, 2, 0, 2, 3, 3, 3, 3, 3, 2, 3, 0, 2, 1, 1, 1, 0, 2, 0, 1, 3, 1, 3, 3, 0, 1, 2, 1, 1, 3, 3, 3, 3, 1, 3, 0, 0, 0, 2, 0, 0, 1, 2, 0, 0, 0, 1, 3, 0, 3, 3, 1, 0, 3, 1, 2, 3, 0, 0, 1, 0, 0, 0, 3, 2, 3, 0, 2, 0, 1, 2, 0, 2, 3, 3, 3, 2, 1]
[1 0 1 2 3 3 1 0 0 3 2 2 0 3 0 2 1 1 1 0 1 1 1 3 2 2 2 2 2 0 3 2 2 2 2 3 1
 0 1 1 3 3 0 2 2 0 3 1 3 2 

In [23]:
# KNN IMPLEMENTATION
neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(RandomX, svm_y)
pred_knn=neigh.predict(test_Xrc)
score=metrics.accuracy_score(Label_OF_test_data, pred_knn)
print(Label_OF_test_data)
print(pred_knn)
print('Accuracy KNN : {}'.format(score))

[3, 2, 1, 0, 2, 3, 1, 0, 1, 3, 3, 2, 2, 0, 2, 3, 1, 1, 1, 2, 0, 1, 1, 3, 2, 1, 0, 2, 2, 0, 0, 2, 2, 2, 2, 3, 2, 3, 1, 1, 3, 3, 3, 2, 2, 3, 3, 1, 2, 2, 3, 3, 2, 1, 0, 1, 3, 2, 3, 2, 3, 2, 1, 1, 1, 0, 1, 2, 2, 0, 2, 0, 0, 2, 1, 3, 0, 3, 1, 1, 0, 2, 3, 0, 3, 3, 3, 1, 1, 1, 2, 3, 2, 0, 3, 3, 2, 0, 2, 0, 2, 0, 3, 3, 1, 1, 3, 0, 3, 0, 1, 0, 1, 1, 3, 3, 0, 1, 3, 0, 2, 3, 2, 2, 3, 3, 3, 3, 3, 2, 2, 1, 2, 1, 1, 0, 0, 2, 3, 1, 1, 0, 0, 1, 2, 0, 3, 0, 2, 1, 2, 3, 2, 0, 3, 2, 2, 1, 0, 3, 2, 1, 0, 0, 0, 1, 2, 2, 3, 0, 0, 0, 0, 1, 0, 0, 2, 1, 0, 2, 2, 1, 3, 1, 3, 2, 0, 3, 1, 0, 0, 2, 2, 0, 1, 3, 2, 1, 0, 0, 3, 3, 0, 1, 2, 0, 1, 3, 2, 1, 2, 2, 1, 3, 1, 0, 1, 2, 2, 2, 1, 3, 2, 0, 2, 3, 3, 3, 3, 3, 2, 3, 0, 2, 1, 1, 1, 0, 2, 0, 1, 3, 1, 3, 3, 0, 1, 2, 1, 1, 3, 3, 3, 3, 1, 3, 0, 0, 0, 2, 0, 0, 1, 2, 0, 0, 0, 1, 3, 0, 3, 3, 1, 0, 3, 1, 2, 3, 0, 0, 1, 0, 0, 0, 3, 2, 3, 0, 2, 0, 1, 2, 0, 2, 3, 3, 3, 2, 1]
[1 2 1 2 2 3 1 0 3 3 3 2 0 2 2 3 1 1 1 2 2 1 1 0 2 2 2 0 2 1 2 2 2 2 2 3 2
 0 1 1 2 1 1 0 2 0 3 1 0 2 

In [24]:
# LinearDiscriminantAnalysis IMPLEMENTATION
clf = LinearDiscriminantAnalysis()
clf.fit(RandomX, svm_y)
pred_LinearDiscriminantAnalysis=clf.predict(test_Xrc)
score=metrics.accuracy_score(Label_OF_test_data, pred_LinearDiscriminantAnalysis)
print(Label_OF_test_data)
print(pred_LinearDiscriminantAnalysis)
print('Accuracy LinearDiscriminantAnalysis : {}'.format(score))


C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


[3, 2, 1, 0, 2, 3, 1, 0, 1, 3, 3, 2, 2, 0, 2, 3, 1, 1, 1, 2, 0, 1, 1, 3, 2, 1, 0, 2, 2, 0, 0, 2, 2, 2, 2, 3, 2, 3, 1, 1, 3, 3, 3, 2, 2, 3, 3, 1, 2, 2, 3, 3, 2, 1, 0, 1, 3, 2, 3, 2, 3, 2, 1, 1, 1, 0, 1, 2, 2, 0, 2, 0, 0, 2, 1, 3, 0, 3, 1, 1, 0, 2, 3, 0, 3, 3, 3, 1, 1, 1, 2, 3, 2, 0, 3, 3, 2, 0, 2, 0, 2, 0, 3, 3, 1, 1, 3, 0, 3, 0, 1, 0, 1, 1, 3, 3, 0, 1, 3, 0, 2, 3, 2, 2, 3, 3, 3, 3, 3, 2, 2, 1, 2, 1, 1, 0, 0, 2, 3, 1, 1, 0, 0, 1, 2, 0, 3, 0, 2, 1, 2, 3, 2, 0, 3, 2, 2, 1, 0, 3, 2, 1, 0, 0, 0, 1, 2, 2, 3, 0, 0, 0, 0, 1, 0, 0, 2, 1, 0, 2, 2, 1, 3, 1, 3, 2, 0, 3, 1, 0, 0, 2, 2, 0, 1, 3, 2, 1, 0, 0, 3, 3, 0, 1, 2, 0, 1, 3, 2, 1, 2, 2, 1, 3, 1, 0, 1, 2, 2, 2, 1, 3, 2, 0, 2, 3, 3, 3, 3, 3, 2, 3, 0, 2, 1, 1, 1, 0, 2, 0, 1, 3, 1, 3, 3, 0, 1, 2, 1, 1, 3, 3, 3, 3, 1, 3, 0, 0, 0, 2, 0, 0, 1, 2, 0, 0, 0, 1, 3, 0, 3, 3, 1, 0, 3, 1, 2, 3, 0, 0, 1, 0, 0, 0, 3, 2, 3, 0, 2, 0, 1, 2, 0, 2, 3, 3, 3, 2, 1]
[2 2 1 3 2 3 3 3 0 2 0 2 3 0 2 2 1 0 1 2 2 0 1 3 3 2 3 2 2 1 3 2 1 3 3 3 0
 0 1 1 3 2 0 0 0 0 2 2 3 2 

In [25]:
# GaussianNB IMPLEMENTATION
clf = GaussianNB()
clf.fit(RandomX, svm_y)
pred_GaussianNB=clf.predict(test_Xrc)
score=metrics.accuracy_score(Label_OF_test_data, pred_GaussianNB)
print(Label_OF_test_data)
print(pred_GaussianNB)
print('Accuracy GaussianNB : {}'.format(score))

[3, 2, 1, 0, 2, 3, 1, 0, 1, 3, 3, 2, 2, 0, 2, 3, 1, 1, 1, 2, 0, 1, 1, 3, 2, 1, 0, 2, 2, 0, 0, 2, 2, 2, 2, 3, 2, 3, 1, 1, 3, 3, 3, 2, 2, 3, 3, 1, 2, 2, 3, 3, 2, 1, 0, 1, 3, 2, 3, 2, 3, 2, 1, 1, 1, 0, 1, 2, 2, 0, 2, 0, 0, 2, 1, 3, 0, 3, 1, 1, 0, 2, 3, 0, 3, 3, 3, 1, 1, 1, 2, 3, 2, 0, 3, 3, 2, 0, 2, 0, 2, 0, 3, 3, 1, 1, 3, 0, 3, 0, 1, 0, 1, 1, 3, 3, 0, 1, 3, 0, 2, 3, 2, 2, 3, 3, 3, 3, 3, 2, 2, 1, 2, 1, 1, 0, 0, 2, 3, 1, 1, 0, 0, 1, 2, 0, 3, 0, 2, 1, 2, 3, 2, 0, 3, 2, 2, 1, 0, 3, 2, 1, 0, 0, 0, 1, 2, 2, 3, 0, 0, 0, 0, 1, 0, 0, 2, 1, 0, 2, 2, 1, 3, 1, 3, 2, 0, 3, 1, 0, 0, 2, 2, 0, 1, 3, 2, 1, 0, 0, 3, 3, 0, 1, 2, 0, 1, 3, 2, 1, 2, 2, 1, 3, 1, 0, 1, 2, 2, 2, 1, 3, 2, 0, 2, 3, 3, 3, 3, 3, 2, 3, 0, 2, 1, 1, 1, 0, 2, 0, 1, 3, 1, 3, 3, 0, 1, 2, 1, 1, 3, 3, 3, 3, 1, 3, 0, 0, 0, 2, 0, 0, 1, 2, 0, 0, 0, 1, 3, 0, 3, 3, 1, 0, 3, 1, 2, 3, 0, 0, 1, 0, 0, 0, 3, 2, 3, 0, 2, 0, 1, 2, 0, 2, 3, 3, 3, 2, 1]
[1 1 1 3 2 3 1 0 1 1 2 0 0 2 0 3 1 0 1 2 2 1 1 2 2 2 2 0 2 0 2 1 0 0 2 2 2
 1 1 1 3 0 1 2 0 0 2 1 1 2 

In [26]:
# DecisionTreeClassifier 
clf = DecisionTreeClassifier(random_state=0)
clf.fit(RandomX, svm_y)
pred_DecisionTreeClassifier=clf.predict(test_Xrc)
score=metrics.accuracy_score(Label_OF_test_data, pred_DecisionTreeClassifier)
print(Label_OF_test_data)
print(pred_DecisionTreeClassifier)
print('Accuracy DecisionTreeClassifier : {}'.format(score))

[3, 2, 1, 0, 2, 3, 1, 0, 1, 3, 3, 2, 2, 0, 2, 3, 1, 1, 1, 2, 0, 1, 1, 3, 2, 1, 0, 2, 2, 0, 0, 2, 2, 2, 2, 3, 2, 3, 1, 1, 3, 3, 3, 2, 2, 3, 3, 1, 2, 2, 3, 3, 2, 1, 0, 1, 3, 2, 3, 2, 3, 2, 1, 1, 1, 0, 1, 2, 2, 0, 2, 0, 0, 2, 1, 3, 0, 3, 1, 1, 0, 2, 3, 0, 3, 3, 3, 1, 1, 1, 2, 3, 2, 0, 3, 3, 2, 0, 2, 0, 2, 0, 3, 3, 1, 1, 3, 0, 3, 0, 1, 0, 1, 1, 3, 3, 0, 1, 3, 0, 2, 3, 2, 2, 3, 3, 3, 3, 3, 2, 2, 1, 2, 1, 1, 0, 0, 2, 3, 1, 1, 0, 0, 1, 2, 0, 3, 0, 2, 1, 2, 3, 2, 0, 3, 2, 2, 1, 0, 3, 2, 1, 0, 0, 0, 1, 2, 2, 3, 0, 0, 0, 0, 1, 0, 0, 2, 1, 0, 2, 2, 1, 3, 1, 3, 2, 0, 3, 1, 0, 0, 2, 2, 0, 1, 3, 2, 1, 0, 0, 3, 3, 0, 1, 2, 0, 1, 3, 2, 1, 2, 2, 1, 3, 1, 0, 1, 2, 2, 2, 1, 3, 2, 0, 2, 3, 3, 3, 3, 3, 2, 3, 0, 2, 1, 1, 1, 0, 2, 0, 1, 3, 1, 3, 3, 0, 1, 2, 1, 1, 3, 3, 3, 3, 1, 3, 0, 0, 0, 2, 0, 0, 1, 2, 0, 0, 0, 1, 3, 0, 3, 3, 1, 0, 3, 1, 2, 3, 0, 0, 1, 0, 0, 0, 3, 2, 3, 0, 2, 0, 1, 2, 0, 2, 3, 3, 3, 2, 1]
[1 3 1 3 2 2 1 0 0 0 0 0 0 2 0 2 1 1 1 3 0 1 1 2 2 2 2 0 2 0 3 3 3 3 2 3 2
 0 1 0 2 0 1 2 3 0 3 1 1 2 

# This part of code is implementing PCA and ml algo

In [27]:
# *********************** PCA + lOGISTIC REGRESSION *****************

from sklearn.decomposition import PCA
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.model_selection import train_test_split
import pandas as pd

In [28]:
RandomX.shape

(1228, 2500)

In [29]:
# Pca_X = []
# Pca_Y = []
Pca_train_X = []
Pca_test__X=[]
Pca_train_X=RandomX
Pca_test_X=test_Xrc
print(Pca_train_X.shape)
print(len(Pca_train_X))
print(Pca_test_X.shape)
print(len(Pca_test_X))

(1228, 2500)
1228
(299, 2500)
299


In [30]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# Fit on training set only.
scaler.fit(Pca_train_X)

# Apply transform to both the training set and the test set.
Pca_train_X = scaler.transform(Pca_train_X)
Pca_test_X = scaler.transform(Pca_test_X)

C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [31]:
from sklearn.decomposition import PCA
pca = PCA(.95)
pca.fit(Pca_train_X)

PCA(copy=True, iterated_power='auto', n_components=0.95, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [28]:
pca.n_components_

148

In [32]:
Pca_train_X = pca.transform(Pca_train_X)
Pca_test_X = pca.transform(Pca_test_X)

In [33]:
from sklearn.linear_model import LogisticRegression

# all parameters not specified are set to their defaults# all pa 
# default solver is incredibly slow thats why we change it
# solver = 'lbfgs'
logisticRegr = LogisticRegression(solver = 'lbfgs')
logisticRegr.fit(Pca_train_X, svm_y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='lbfgs', tol=0.0001,
          verbose=0, warm_start=False)

In [34]:
pred_logReg=logisticRegr.predict(Pca_test_X)

In [35]:
score=metrics.accuracy_score(Label_OF_test_data, pred_logReg)
print(Label_OF_test_data)
print(pred_logReg)
print('Accuracy PCA + LogisticRegression : {}'.format(score))

[3, 2, 1, 0, 2, 3, 1, 0, 1, 3, 3, 2, 2, 0, 2, 3, 1, 1, 1, 2, 0, 1, 1, 3, 2, 1, 0, 2, 2, 0, 0, 2, 2, 2, 2, 3, 2, 3, 1, 1, 3, 3, 3, 2, 2, 3, 3, 1, 2, 2, 3, 3, 2, 1, 0, 1, 3, 2, 3, 2, 3, 2, 1, 1, 1, 0, 1, 2, 2, 0, 2, 0, 0, 2, 1, 3, 0, 3, 1, 1, 0, 2, 3, 0, 3, 3, 3, 1, 1, 1, 2, 3, 2, 0, 3, 3, 2, 0, 2, 0, 2, 0, 3, 3, 1, 1, 3, 0, 3, 0, 1, 0, 1, 1, 3, 3, 0, 1, 3, 0, 2, 3, 2, 2, 3, 3, 3, 3, 3, 2, 2, 1, 2, 1, 1, 0, 0, 2, 3, 1, 1, 0, 0, 1, 2, 0, 3, 0, 2, 1, 2, 3, 2, 0, 3, 2, 2, 1, 0, 3, 2, 1, 0, 0, 0, 1, 2, 2, 3, 0, 0, 0, 0, 1, 0, 0, 2, 1, 0, 2, 2, 1, 3, 1, 3, 2, 0, 3, 1, 0, 0, 2, 2, 0, 1, 3, 2, 1, 0, 0, 3, 3, 0, 1, 2, 0, 1, 3, 2, 1, 2, 2, 1, 3, 1, 0, 1, 2, 2, 2, 1, 3, 2, 0, 2, 3, 3, 3, 3, 3, 2, 3, 0, 2, 1, 1, 1, 0, 2, 0, 1, 3, 1, 3, 3, 0, 1, 2, 1, 1, 3, 3, 3, 3, 1, 3, 0, 0, 0, 2, 0, 0, 1, 2, 0, 0, 0, 1, 3, 0, 3, 3, 1, 0, 3, 1, 2, 3, 0, 0, 1, 0, 0, 0, 3, 2, 3, 0, 2, 0, 1, 2, 0, 2, 3, 3, 3, 2, 1]
[1 0 1 3 2 2 1 0 2 3 0 2 0 3 0 3 1 1 1 3 1 1 1 3 2 2 1 3 2 0 3 2 2 2 2 2 2
 0 1 1 3 3 0 2 2 0 3 1 3 2 

In [36]:
# **************************** PCA + SVM *****************
s_v_m.fit(Pca_train_X,svm_y)
pred_svm_pca=s_v_m.predict(Pca_test_X)
# print(pred_rc)
#  accuracy scheck svm

score=metrics.accuracy_score(Label_OF_test_data, pred_svm_pca)
print(Label_OF_test_data)
print(pred_svm_pca)
print('Accuracy PCA + SVM: {}'.format(score))

[3, 2, 1, 0, 2, 3, 1, 0, 1, 3, 3, 2, 2, 0, 2, 3, 1, 1, 1, 2, 0, 1, 1, 3, 2, 1, 0, 2, 2, 0, 0, 2, 2, 2, 2, 3, 2, 3, 1, 1, 3, 3, 3, 2, 2, 3, 3, 1, 2, 2, 3, 3, 2, 1, 0, 1, 3, 2, 3, 2, 3, 2, 1, 1, 1, 0, 1, 2, 2, 0, 2, 0, 0, 2, 1, 3, 0, 3, 1, 1, 0, 2, 3, 0, 3, 3, 3, 1, 1, 1, 2, 3, 2, 0, 3, 3, 2, 0, 2, 0, 2, 0, 3, 3, 1, 1, 3, 0, 3, 0, 1, 0, 1, 1, 3, 3, 0, 1, 3, 0, 2, 3, 2, 2, 3, 3, 3, 3, 3, 2, 2, 1, 2, 1, 1, 0, 0, 2, 3, 1, 1, 0, 0, 1, 2, 0, 3, 0, 2, 1, 2, 3, 2, 0, 3, 2, 2, 1, 0, 3, 2, 1, 0, 0, 0, 1, 2, 2, 3, 0, 0, 0, 0, 1, 0, 0, 2, 1, 0, 2, 2, 1, 3, 1, 3, 2, 0, 3, 1, 0, 0, 2, 2, 0, 1, 3, 2, 1, 0, 0, 3, 3, 0, 1, 2, 0, 1, 3, 2, 1, 2, 2, 1, 3, 1, 0, 1, 2, 2, 2, 1, 3, 2, 0, 2, 3, 3, 3, 3, 3, 2, 3, 0, 2, 1, 1, 1, 0, 2, 0, 1, 3, 1, 3, 3, 0, 1, 2, 1, 1, 3, 3, 3, 3, 1, 3, 0, 0, 0, 2, 0, 0, 1, 2, 0, 0, 0, 1, 3, 0, 3, 3, 1, 0, 3, 1, 2, 3, 0, 0, 1, 0, 0, 0, 3, 2, 3, 0, 2, 0, 1, 2, 0, 2, 3, 3, 3, 2, 1]
[0 2 1 3 2 3 1 0 3 3 2 2 0 3 2 3 1 2 1 3 0 1 1 3 2 2 2 2 2 0 3 2 2 2 2 3 2
 3 1 1 2 3 3 2 2 0 3 1 0 2 

In [37]:
# PCA + RC
clf = RandomForestClassifier(n_estimators=100)
clf.fit(Pca_train_X,svm_y)
pred_rc=clf.predict(Pca_test_X)
score=metrics.accuracy_score(Label_OF_test_data, pred_rc)
print(Label_OF_test_data)
print(pred_rc)
print('Accuracy PCA + RANDOM CLASSIFIER: {}'.format(score))

[3, 2, 1, 0, 2, 3, 1, 0, 1, 3, 3, 2, 2, 0, 2, 3, 1, 1, 1, 2, 0, 1, 1, 3, 2, 1, 0, 2, 2, 0, 0, 2, 2, 2, 2, 3, 2, 3, 1, 1, 3, 3, 3, 2, 2, 3, 3, 1, 2, 2, 3, 3, 2, 1, 0, 1, 3, 2, 3, 2, 3, 2, 1, 1, 1, 0, 1, 2, 2, 0, 2, 0, 0, 2, 1, 3, 0, 3, 1, 1, 0, 2, 3, 0, 3, 3, 3, 1, 1, 1, 2, 3, 2, 0, 3, 3, 2, 0, 2, 0, 2, 0, 3, 3, 1, 1, 3, 0, 3, 0, 1, 0, 1, 1, 3, 3, 0, 1, 3, 0, 2, 3, 2, 2, 3, 3, 3, 3, 3, 2, 2, 1, 2, 1, 1, 0, 0, 2, 3, 1, 1, 0, 0, 1, 2, 0, 3, 0, 2, 1, 2, 3, 2, 0, 3, 2, 2, 1, 0, 3, 2, 1, 0, 0, 0, 1, 2, 2, 3, 0, 0, 0, 0, 1, 0, 0, 2, 1, 0, 2, 2, 1, 3, 1, 3, 2, 0, 3, 1, 0, 0, 2, 2, 0, 1, 3, 2, 1, 0, 0, 3, 3, 0, 1, 2, 0, 1, 3, 2, 1, 2, 2, 1, 3, 1, 0, 1, 2, 2, 2, 1, 3, 2, 0, 2, 3, 3, 3, 3, 3, 2, 3, 0, 2, 1, 1, 1, 0, 2, 0, 1, 3, 1, 3, 3, 0, 1, 2, 1, 1, 3, 3, 3, 3, 1, 3, 0, 0, 0, 2, 0, 0, 1, 2, 0, 0, 0, 1, 3, 0, 3, 3, 1, 0, 3, 1, 2, 3, 0, 0, 1, 0, 0, 0, 3, 2, 3, 0, 2, 0, 1, 2, 0, 2, 3, 3, 3, 2, 1]
[2 0 1 0 2 2 1 0 2 2 0 2 0 2 2 2 1 0 1 0 2 1 1 3 2 2 0 2 2 3 2 2 2 2 2 3 2
 2 1 1 0 3 2 2 2 2 3 2 0 2 

In [38]:
# PCA + KNN IMPLEMENTATION
neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(Pca_train_X, svm_y)
pred_knn=neigh.predict(Pca_test_X)
score=metrics.accuracy_score(Label_OF_test_data, pred_knn)
print(Label_OF_test_data)
print(pred_knn)
print('Accuracy PCA + KNN : {}'.format(score))

[3, 2, 1, 0, 2, 3, 1, 0, 1, 3, 3, 2, 2, 0, 2, 3, 1, 1, 1, 2, 0, 1, 1, 3, 2, 1, 0, 2, 2, 0, 0, 2, 2, 2, 2, 3, 2, 3, 1, 1, 3, 3, 3, 2, 2, 3, 3, 1, 2, 2, 3, 3, 2, 1, 0, 1, 3, 2, 3, 2, 3, 2, 1, 1, 1, 0, 1, 2, 2, 0, 2, 0, 0, 2, 1, 3, 0, 3, 1, 1, 0, 2, 3, 0, 3, 3, 3, 1, 1, 1, 2, 3, 2, 0, 3, 3, 2, 0, 2, 0, 2, 0, 3, 3, 1, 1, 3, 0, 3, 0, 1, 0, 1, 1, 3, 3, 0, 1, 3, 0, 2, 3, 2, 2, 3, 3, 3, 3, 3, 2, 2, 1, 2, 1, 1, 0, 0, 2, 3, 1, 1, 0, 0, 1, 2, 0, 3, 0, 2, 1, 2, 3, 2, 0, 3, 2, 2, 1, 0, 3, 2, 1, 0, 0, 0, 1, 2, 2, 3, 0, 0, 0, 0, 1, 0, 0, 2, 1, 0, 2, 2, 1, 3, 1, 3, 2, 0, 3, 1, 0, 0, 2, 2, 0, 1, 3, 2, 1, 0, 0, 3, 3, 0, 1, 2, 0, 1, 3, 2, 1, 2, 2, 1, 3, 1, 0, 1, 2, 2, 2, 1, 3, 2, 0, 2, 3, 3, 3, 3, 3, 2, 3, 0, 2, 1, 1, 1, 0, 2, 0, 1, 3, 1, 3, 3, 0, 1, 2, 1, 1, 3, 3, 3, 3, 1, 3, 0, 0, 0, 2, 0, 0, 1, 2, 0, 0, 0, 1, 3, 0, 3, 3, 1, 0, 3, 1, 2, 3, 0, 0, 1, 0, 0, 0, 3, 2, 3, 0, 2, 0, 1, 2, 0, 2, 3, 3, 3, 2, 1]
[1 2 1 2 2 3 1 0 2 3 0 2 0 2 2 3 1 2 1 2 2 1 1 0 2 2 2 0 2 1 3 2 2 2 2 3 2
 0 1 1 2 1 1 2 2 0 3 1 3 2 

In [39]:
# PCA + LinearDiscriminantAnalysis IMPLEMENTATION
clf = LinearDiscriminantAnalysis()
clf.fit(Pca_train_X, svm_y)
pred_LinearDiscriminantAnalysis=clf.predict(Pca_test_X)
score=metrics.accuracy_score(Label_OF_test_data, pred_LinearDiscriminantAnalysis)
print(Label_OF_test_data)
print(pred_LinearDiscriminantAnalysis)
print('Accuracy PCA + LinearDiscriminantAnalysis : {}'.format(score))


[3, 2, 1, 0, 2, 3, 1, 0, 1, 3, 3, 2, 2, 0, 2, 3, 1, 1, 1, 2, 0, 1, 1, 3, 2, 1, 0, 2, 2, 0, 0, 2, 2, 2, 2, 3, 2, 3, 1, 1, 3, 3, 3, 2, 2, 3, 3, 1, 2, 2, 3, 3, 2, 1, 0, 1, 3, 2, 3, 2, 3, 2, 1, 1, 1, 0, 1, 2, 2, 0, 2, 0, 0, 2, 1, 3, 0, 3, 1, 1, 0, 2, 3, 0, 3, 3, 3, 1, 1, 1, 2, 3, 2, 0, 3, 3, 2, 0, 2, 0, 2, 0, 3, 3, 1, 1, 3, 0, 3, 0, 1, 0, 1, 1, 3, 3, 0, 1, 3, 0, 2, 3, 2, 2, 3, 3, 3, 3, 3, 2, 2, 1, 2, 1, 1, 0, 0, 2, 3, 1, 1, 0, 0, 1, 2, 0, 3, 0, 2, 1, 2, 3, 2, 0, 3, 2, 2, 1, 0, 3, 2, 1, 0, 0, 0, 1, 2, 2, 3, 0, 0, 0, 0, 1, 0, 0, 2, 1, 0, 2, 2, 1, 3, 1, 3, 2, 0, 3, 1, 0, 0, 2, 2, 0, 1, 3, 2, 1, 0, 0, 3, 3, 0, 1, 2, 0, 1, 3, 2, 1, 2, 2, 1, 3, 1, 0, 1, 2, 2, 2, 1, 3, 2, 0, 2, 3, 3, 3, 3, 3, 2, 3, 0, 2, 1, 1, 1, 0, 2, 0, 1, 3, 1, 3, 3, 0, 1, 2, 1, 1, 3, 3, 3, 3, 1, 3, 0, 0, 0, 2, 0, 0, 1, 2, 0, 0, 0, 1, 3, 0, 3, 3, 1, 0, 3, 1, 2, 3, 0, 0, 1, 0, 0, 0, 3, 2, 3, 0, 2, 0, 1, 2, 0, 2, 3, 3, 3, 2, 1]
[1 0 1 3 2 2 1 0 2 3 0 2 0 3 0 3 1 1 1 2 2 1 1 3 2 2 2 3 2 0 3 2 2 2 2 2 2
 0 1 1 2 3 0 2 3 0 3 1 2 2 

In [40]:
# PCA + GaussianNB IMPLEMENTATION
clf = GaussianNB()
clf.fit(Pca_train_X, svm_y)
pred_GaussianNB=clf.predict(Pca_test_X)
score=metrics.accuracy_score(Label_OF_test_data, pred_GaussianNB)
print(Label_OF_test_data)
print(pred_GaussianNB)
print('Accuracy PCA + GaussianNB : {}'.format(score))

[3, 2, 1, 0, 2, 3, 1, 0, 1, 3, 3, 2, 2, 0, 2, 3, 1, 1, 1, 2, 0, 1, 1, 3, 2, 1, 0, 2, 2, 0, 0, 2, 2, 2, 2, 3, 2, 3, 1, 1, 3, 3, 3, 2, 2, 3, 3, 1, 2, 2, 3, 3, 2, 1, 0, 1, 3, 2, 3, 2, 3, 2, 1, 1, 1, 0, 1, 2, 2, 0, 2, 0, 0, 2, 1, 3, 0, 3, 1, 1, 0, 2, 3, 0, 3, 3, 3, 1, 1, 1, 2, 3, 2, 0, 3, 3, 2, 0, 2, 0, 2, 0, 3, 3, 1, 1, 3, 0, 3, 0, 1, 0, 1, 1, 3, 3, 0, 1, 3, 0, 2, 3, 2, 2, 3, 3, 3, 3, 3, 2, 2, 1, 2, 1, 1, 0, 0, 2, 3, 1, 1, 0, 0, 1, 2, 0, 3, 0, 2, 1, 2, 3, 2, 0, 3, 2, 2, 1, 0, 3, 2, 1, 0, 0, 0, 1, 2, 2, 3, 0, 0, 0, 0, 1, 0, 0, 2, 1, 0, 2, 2, 1, 3, 1, 3, 2, 0, 3, 1, 0, 0, 2, 2, 0, 1, 3, 2, 1, 0, 0, 3, 3, 0, 1, 2, 0, 1, 3, 2, 1, 2, 2, 1, 3, 1, 0, 1, 2, 2, 2, 1, 3, 2, 0, 2, 3, 3, 3, 3, 3, 2, 3, 0, 2, 1, 1, 1, 0, 2, 0, 1, 3, 1, 3, 3, 0, 1, 2, 1, 1, 3, 3, 3, 3, 1, 3, 0, 0, 0, 2, 0, 0, 1, 2, 0, 0, 0, 1, 3, 0, 3, 3, 1, 0, 3, 1, 2, 3, 0, 0, 1, 0, 0, 0, 3, 2, 3, 0, 2, 0, 1, 2, 0, 2, 3, 3, 3, 2, 1]
[0 1 1 3 1 2 1 0 3 2 1 1 0 3 0 3 1 0 1 3 1 1 1 2 2 1 1 2 1 2 3 2 1 1 2 1 1
 0 1 1 0 0 0 2 2 2 2 1 1 2 

In [42]:
# PCA + DecisionTreeClassifier 
clf = DecisionTreeClassifier(random_state=0)
clf.fit(Pca_train_X, svm_y)
pred_DecisionTreeClassifier=clf.predict(Pca_test_X)
score=metrics.accuracy_score(Label_OF_test_data, pred_DecisionTreeClassifier)
print(Label_OF_test_data)
print(pred_DecisionTreeClassifier)
print('Accuracy pca + DecisionTreeClassifier : {}'.format(score))

[3, 2, 1, 0, 2, 3, 1, 0, 1, 3, 3, 2, 2, 0, 2, 3, 1, 1, 1, 2, 0, 1, 1, 3, 2, 1, 0, 2, 2, 0, 0, 2, 2, 2, 2, 3, 2, 3, 1, 1, 3, 3, 3, 2, 2, 3, 3, 1, 2, 2, 3, 3, 2, 1, 0, 1, 3, 2, 3, 2, 3, 2, 1, 1, 1, 0, 1, 2, 2, 0, 2, 0, 0, 2, 1, 3, 0, 3, 1, 1, 0, 2, 3, 0, 3, 3, 3, 1, 1, 1, 2, 3, 2, 0, 3, 3, 2, 0, 2, 0, 2, 0, 3, 3, 1, 1, 3, 0, 3, 0, 1, 0, 1, 1, 3, 3, 0, 1, 3, 0, 2, 3, 2, 2, 3, 3, 3, 3, 3, 2, 2, 1, 2, 1, 1, 0, 0, 2, 3, 1, 1, 0, 0, 1, 2, 0, 3, 0, 2, 1, 2, 3, 2, 0, 3, 2, 2, 1, 0, 3, 2, 1, 0, 0, 0, 1, 2, 2, 3, 0, 0, 0, 0, 1, 0, 0, 2, 1, 0, 2, 2, 1, 3, 1, 3, 2, 0, 3, 1, 0, 0, 2, 2, 0, 1, 3, 2, 1, 0, 0, 3, 3, 0, 1, 2, 0, 1, 3, 2, 1, 2, 2, 1, 3, 1, 0, 1, 2, 2, 2, 1, 3, 2, 0, 2, 3, 3, 3, 3, 3, 2, 3, 0, 2, 1, 1, 1, 0, 2, 0, 1, 3, 1, 3, 3, 0, 1, 2, 1, 1, 3, 3, 3, 3, 1, 3, 0, 0, 0, 2, 0, 0, 1, 2, 0, 0, 0, 1, 3, 0, 3, 3, 1, 0, 3, 1, 2, 3, 0, 0, 1, 0, 0, 0, 3, 2, 3, 0, 2, 0, 1, 2, 0, 2, 3, 3, 3, 2, 1]
[0 1 1 0 2 3 1 0 3 0 2 3 0 3 1 2 1 1 1 3 1 1 1 3 3 2 0 0 3 3 3 2 2 2 2 2 2
 0 1 1 3 0 3 3 3 0 3 1 2 2 

# This is for LBP and Ml algo

In [43]:
from skimage import feature
# 
from skimage.feature import local_binary_pattern
from sklearn.svm import LinearSVC

In [46]:
# loading the training data
hist_train=[]
label_b4 = []
hist_train_label=[]
for img in tqdm(os.listdir(TRAIN_DIR)):

    # labeling the images
    label = label_img(img)
    label_b4.append(label)
    path = os.path.join(TRAIN_DIR, img)

    # loading the image from the path and then converting them into
    # greyscale for easier covnet prob
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)

    # resizing the image for processing them in the covnet
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    lbp = feature.local_binary_pattern(img, 24,8, method="uniform")
    (hist, _) = np.histogram(lbp.ravel(),
        bins=np.arange(0, 24+ 3),
        range=(0, 24+2))

    # normalize the histogram
    hist = hist.astype("float")
    hist /= (hist.sum() + 1e-7)
    hist_train.append(hist)
        # final step-forming the training data list with numpy array of the images
#         training_data.append([np.array(img), np.array(label)])
 
#     # shuffling of the training data to preserve the random state of our data
#     shuffle(training_data)
 
#     # saving our trained data for further uses if required
#     np.save('train_data.npy', training_data)
hist_train[0:5]
for val in label_b4:
    if np.argmax(val) == 0:
#         str_label ='Happy'
        hist_train_label.append(0)
    elif np.argmax(val) == 1:
#         str_label ='sad'
        hist_train_label.append(1)
    elif np.argmax(val) == 2:
#         str_label ='Angry'
        hist_train_label.append(2)
    elif np.argmax(val) == 3:
#         str_label ='normal'
        hist_train_label.append(3)

# svm_y[0:27]

100%|██████████████████████████████████████| 1228/1228 [00:13<00:00, 94.24it/s]


In [47]:
hist_train_label[1227]

1

In [48]:
model = LinearSVC(C=100.0, random_state=42)
model.fit(hist_train, hist_train_label)

LinearSVC(C=100.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=42, tol=0.0001,
     verbose=0)

In [49]:
# loading testing data 
hist_test=[]
label_b4 = []
hist_test_label=[]
for img in tqdm(os.listdir(TEST_DIR)):

    # labeling the images
    label = label_img(img)
    label_b4.append(label)
    path = os.path.join(TEST_DIR, img)

    # loading the image from the path and then converting them into
    # greyscale for easier covnet prob
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)

    # resizing the image for processing them in the covnet
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    lbp = feature.local_binary_pattern(img, 24,8, method="uniform")
    (hist, _) = np.histogram(lbp.ravel(),
        bins=np.arange(0, 24+ 3),
        range=(0, 24+2))

    # normalize the histogram
    hist = hist.astype("float")
    hist /= (hist.sum() + 1e-7)
    hist_test.append(hist)
        # final step-forming the training data list with numpy array of the images
#         training_data.append([np.array(img), np.array(label)])
 
#     # shuffling of the training data to preserve the random state of our data
#     shuffle(training_data)
 
#     # saving our trained data for further uses if required
#     np.save('train_data.npy', training_data)
print(hist_test[0:5])
for val in label_b4:
    if np.argmax(val) == 0:
#         str_label ='Happy'
        hist_test_label.append(0)
    elif np.argmax(val) == 1:
#         str_label ='sad'
        hist_test_label.append(1)
    elif np.argmax(val) == 2:
#         str_label ='Angry'
        hist_test_label.append(2)
    elif np.argmax(val) == 3:
#         str_label ='Angry'
        hist_test_label.append(3)
hist_test_label

100%|████████████████████████████████████████| 299/299 [00:03<00:00, 85.81it/s]


[array([0.0588, 0.0548, 0.0404, 0.0204, 0.0172, 0.0248, 0.0164, 0.0116,
       0.0212, 0.01  , 0.0132, 0.018 , 0.0104, 0.0224, 0.0104, 0.018 ,
       0.0052, 0.0112, 0.0032, 0.006 , 0.0024, 0.0084, 0.008 , 0.0172,
       0.0208, 0.5496]), array([0.0792, 0.042 , 0.0336, 0.0156, 0.0096, 0.0096, 0.0132, 0.0112,
       0.01  , 0.0088, 0.004 , 0.0068, 0.004 , 0.0084, 0.004 , 0.0144,
       0.0068, 0.0172, 0.0084, 0.0136, 0.0064, 0.0072, 0.01  , 0.0092,
       0.0288, 0.618 ]), array([0.0776, 0.0488, 0.036 , 0.0156, 0.0092, 0.0092, 0.0136, 0.0132,
       0.0076, 0.0068, 0.0036, 0.0044, 0.0036, 0.0064, 0.0032, 0.0148,
       0.0048, 0.0168, 0.0104, 0.0124, 0.0064, 0.0116, 0.0088, 0.0128,
       0.0256, 0.6168]), array([0.0696, 0.0408, 0.0312, 0.0176, 0.01  , 0.0108, 0.0196, 0.0152,
       0.0148, 0.0128, 0.0092, 0.0056, 0.006 , 0.0096, 0.0068, 0.0148,
       0.0076, 0.0184, 0.0096, 0.0096, 0.006 , 0.0064, 0.0072, 0.0104,
       0.0292, 0.6012]), array([0.0696, 0.0596, 0.0336, 0.0172, 0.012 , 

[2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [51]:
prediction = model.predict(hist_test)

score=metrics.accuracy_score(hist_test_label, prediction)
print(hist_test_label)
print(prediction)
print('Accuracy LinearSVC: {}'.format(score))

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0 2 2 2 2 2 0 1 0 2 2 2 0 2 0 3 2 2 2 3 3 3 3 3 3 3 2 2 2 2 2 1 1 1 2 2 1
 2 2 2 2 2 2 2 3 2 2 0 0 0 

In [52]:
# LBP + RC
clf = RandomForestClassifier(n_estimators=100)
clf.fit(hist_train,hist_train_label)
pred_rc=clf.predict(hist_test)
score=metrics.accuracy_score(hist_test_label, pred_rc)
print(hist_test_label)
print(pred_rc)
print('Accuracy LBP + RANDOM CLASSIFIER: {}'.format(score))

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[3 2 2 2 2 2 0 0 0 0 2 2 0 3 0 3 3 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 1 0 2 3 1
 2 2 2 2 2 3 2 2 2 2 3 3 1 

In [53]:
# LBP + KNN IMPLEMENTATION
neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(hist_train, hist_train_label)
pred_knn=neigh.predict(hist_test)
score=metrics.accuracy_score(hist_test_label, pred_knn)
print(hist_test_label)
print(pred_knn)
print('Accuracy LBP + KNN : {}'.format(score))

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1 0 0 3 2 0 0 0 0 3 0 2 1 2 0 0 1 2 3 2 2 2 2 2 2 2 2 2 2 2 0 1 2 0 2 2 2
 2 2 0 2 2 2 2 2 2 2 0 3 0 

In [55]:
# LBP + LinearDiscriminantAnalysis IMPLEMENTATION
clf = LinearDiscriminantAnalysis()
clf.fit(hist_train, hist_train_label)
pred_LinearDiscriminantAnalysis=clf.predict(hist_test)
score=metrics.accuracy_score(hist_test_label, pred_LinearDiscriminantAnalysis)
print(hist_test_label)
print(pred_LinearDiscriminantAnalysis)
print('Accuracy LBP + LinearDiscriminantAnalysis : {}'.format(score))

C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0 2 2 2 2 2 0 1 0 2 2 2 0 2 0 3 3 2 2 3 3 3 3 3 3 3 2 2 2 2 2 1 1 1 2 2 1
 2 2 2 2 1 2 3 3 2 1 0 0 0 

In [56]:
# LBP + GaussianNB IMPLEMENTATION
clf = GaussianNB()
clf.fit(hist_train, hist_train_label)
pred_GaussianNB=clf.predict(hist_test)
score=metrics.accuracy_score(hist_test_label, pred_GaussianNB)
print(hist_test_label)
print(pred_GaussianNB)
print('Accuracy LBP + GaussianNB : {}'.format(score))

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1 2 2 2 2 2 2 1 1 2 2 1 3 2 0 1 2 1 3 3 3 2 2 2 2 3 2 2 2 2 2 1 1 3 2 3 2
 2 2 2 2 1 2 3 3 3 1 0 3 1 

In [58]:
# LBP + DecisionTreeClassifier 
clf = DecisionTreeClassifier(random_state=0)
clf.fit(hist_train, hist_train_label)
pred_DecisionTreeClassifier=clf.predict(hist_test)
score=metrics.accuracy_score(hist_test_label, pred_DecisionTreeClassifier)
print(hist_test_label)
print(pred_DecisionTreeClassifier)
print('Accuracy LBP + DecisionTreeClassifier : {}'.format(score))

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[2 3 2 3 2 0 0 1 0 2 3 2 0 2 0 3 3 3 2 3 2 3 3 2 2 2 2 2 3 2 2 1 1 0 1 3 1
 3 1 3 2 2 0 2 2 2 2 3 3 2 